# mmColorTarget Dependencies

[colour-science.org](https://www.colour-science.org) - March 23, 2021

> Note: The generated JSON file is only compatible with mmColorTarget v3.0 or newer

In [30]:
import json
import colour
import datetime
import numpy as np
from collections import OrderedDict


def get_matrices_and_whitepoints():
    matrices = {}
    whitepoint_names = {}
    for name, colourspace in colour.RGB_COLOURSPACES.items():
        if name in ("aces", "adobe1998", "prophoto"):
            continue
        if not colourspace.use_derived_matrix_XYZ_to_RGB:
            matrices[name] = colourspace.matrix_XYZ_to_RGB.tolist()
        else:
            matrices[name] = colourspace.matrix_XYZ_to_RGB.tolist()
            print ("colorspace {} uses derived".format(name))
        whitepoint_names[name] = colourspace.whitepoint_name
        
    return matrices, whitepoint_names

def get_cat_matrices(whitepoint_names, twp="D65"):
    cat_matrices = {}
    for name in whitepoint_names:
        cat_matrices["{}_to_{}".format(twp, name)] = colour.adaptation.matrix_chromatic_adaptation_VonKries(
            colour.xy_to_XYZ(colour.CCS_ILLUMINANTS["cie_2_1931"][twp]),
            colour.xy_to_XYZ(colour.CCS_ILLUMINANTS["cie_2_1931"][name]),
            transform="CAT02"
        ).tolist()
    return cat_matrices

MATRICES, WHITEPOINTS = get_matrices_and_whitepoints()
UNIQUE_WHITEPOINTS = list(set(WHITEPOINTS.values()))
SRGB_COLOURSPACE = colour.RGB_COLOURSPACES.get("sRGB")
JSON_DATA = {
    "info": "Data generated by colour-science.org",
    "colour_version": colour.__version__,
    "date": datetime.date.today().strftime("%d/%m/%y"),
    "sRGB_to_XYZ_matrix": 
        SRGB_COLOURSPACE.matrix_RGB_to_XYZ.tolist(),
    "XYZ_to_RGB_matrices": MATRICES,
    "whitepoints": WHITEPOINTS,
    "CAT_matrices": get_cat_matrices(UNIQUE_WHITEPOINTS, SRGB_COLOURSPACE.whitepoint_name)
}

with open("../resources/mmColorTarget_colorspace_transforms.json", "w") as json_file:
    json.dump(JSON_DATA, json_file, indent=4)